In [29]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from Data_Loader import load_hist_measurements
from timeit import default_timer as timer
from haversine import haversine_vector
start = timer()


In [50]:
hist_measure_df = load_hist_measurements()

acc_df = pd.read_csv('assets/01_2_SLF_hist_avalanche_accidents_mapped.csv', sep=',', skiprows=0)

In [34]:
import numpy as np
import pandas as pd

def calculate_distance(lt1, ln1, lt2, ln2):
    R = 6373.0  
    lat1 = np.deg2rad(lt1)
    lon1 = np.deg2rad(ln1)
    lat2 = np.deg2rad(lt2)
    lon2 = np.deg2rad(ln2)
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

def generate_data(rows):
    df = pd.DataFrame(columns =['lat', 'lon'])
    df['lat'] = np.random.randint(9999999, 99999999, rows)/1000000
    df['lon'] = np.random.randint(9999999, 99999999, rows)/1000000
    return df

df = generate_data(10000)

latitude = 11.111111
longitude = 121.222222
calculate_distance(
    latitude, longitude,
    df['lat'].values, df['lon'].values)

array([ 5257.32258292,  8735.92510948, 10617.38140339, ...,
        9836.00851   ,  5638.14827281,  8556.46950381])

In [51]:
def calc_avg_air_temp(row):
    radius = 10 # km
    date = row["date"]
    latitude = row["start_zone_coordinates_latitude"]
    longitude = row["start_zone_coordinates_longitude"]
    filtered = hist_measure_df[hist_measure_df['measure_date'] == date]
    result = filtered[calculate_distance(latitude, longitude, filtered['imis_latitude'].values, filtered['imis_longitude'].values) <= radius]
    row["air_temp_mean_stations"] = result['air_temp_day_mean'].mean()
    row["wind_speed_mean_stations"] = result['wind_speed_day_mean'].mean()
    row["snow_surf_temp_day_mean_stations"] = result['snow_surf_temp_day_mean'].mean()
    row["snow_ground_temp_day_mean_stations"] = result['snow_ground_temp_day_mean'].mean()
    row["wind_speed_day_max_stations"] = result['wind_speed_day_max'].mean()
    return row

acc_df = acc_df.apply(calc_avg_air_temp, axis=1)
print(acc_df["wind_speed_mean_stations"])


0       1.429167
1       4.382500
2       0.493478
3       0.260000
4            NaN
          ...   
3296    0.760417
3297    1.220833
3298         NaN
3299    1.548611
3300    1.272917
Name: wind_speed_mean_stations, Length: 3301, dtype: float64


In [53]:
#print(acc_df)
acc_df.to_csv('assets/01_4_SLF_hist_mapped_avalanche_accidents.csv', index=False)

In [7]:
# Vectorized haversine function from: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [26]:
radius = 8 # km
latitude = 46.8250694274902
longitude = 9.7903528213501
date = "2019-01-01"
result = hist_measure_df[haversine(latitude, longitude, hist_measure_df['imis_latitude'], hist_measure_df['imis_longitude']) <= radius]
result = result[result["measure_date"] == date]
air_temperature = result["air_temp_day_mean"].mean()
print(air_temperature)


-2.9270833333333335


In [32]:
radius = 8 # km

for i, accident in acc_df.iterrows():
    latitude = accident['start_zone_coordinates_latitude']
    longitude = accident['start_zone_coordinates_longitude']
    date = accident['date']
    result = hist_measure_df[haversine(latitude, longitude, hist_measure_df['imis_latitude'], hist_measure_df['imis_longitude']) <= radius]
    result = result[result["measure_date"] == date]
    print(len(result))
    #air_temperature = result["air_temp_day_mean"].mean()
    #print(air_temperature)

2
3
0
1
0
0
2
0
2
1
0
0
0
2
2
2
2
2
0
2
2
0
0
1
0
0
0
1
2
0
0
0
0
1
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
1
2
2
0
2
0
0
3
3
0
0
1
1
3
3
2
2
0
1
0
2
1
0
1
1
3
3
3
1
1
0
1
1
1
0
3
1
0
1
0
2
0
0
0
3
3
3
0
0
1
2
0
0
0
2
2
0
0
0
0
2
2
0
0
3
1
1
0
2
0
0
0
0
0
2
2
0
1
0
3
0
0
0
3
0
0
3
2
2
0
2
0
2
2
0
1
2
3
0
1
2
3
4
3
2
3
2
0
2
3
2
0
0
2
1
0
2
0
2
0
1
2
3
2
0
2
2
2
3
0
1
1
0
0
2
1
0
2
3
3
2
4
0
1
1
1
0
0
2
3
2
0
2
2
0
0
2
1
0
0
2
0
1
0
3
1
0
0
0
2
2
1
0
4
2
2
0
2
1
0
0
2
1
1
2
2
1
0
0
2
1
1
1
0
0
0
2
1
0
0
3
1
3
3
0
2
0
0
0
1
1
0
2
2
2
2
0
1
2
0
1
1
2
0
2
0
2
0
0
2
0
0
0
2
0
0
0
0
0
2
0
0
2
0
2
0
2
2
3
2
0
2
1
0
0
2
2
0
1
0
1
0
0
0
2
2
2
0
3
0
2
0
0
1
0
2
2
0
2
1
0
2
3
2
0
0
0
1
3
0
2
2
1
1
2
0
1
2
0
3
2
0
2
3
0
0
0
1
0
2
2
2
2
2
1
2
1
0
1
2
0
1
0
0
0
0
0
3
2
2
0
1
0
2
2
1
1
2
3
0
0
0
1
2
1
2
0
1
2
1
1
1
3
2
1
2
2
1
2
2
1
2
0
2
3
2
1
1
0
2
1
2
2
3
2
0
2
1
0
2
2
0
2
0
2
1
2
2
1
2
2
2
2
1
2
1
3
1
0
1
1
1
2
0
1
1
0
3
0
0
0
0
1
2
4
2
2
2
0
2
2
1
1
1
0
0
1
0
3
3
1
0
2
0
0
3
1
2
1
2
3
2
2
0
1
1
1
3
2
2


KeyboardInterrupt: 

In [3]:
def find_closest_weather_stations(acc_df, hist_measure_df, lat_lon_weight, elevation_weight):
    # Define the weather columns:
    weather_columns = {
        'mean_air_temp': 'air_temp_day_mean',
        'mean_wind_speed': 'wind_speed_day_mean',
        'max_wind_speed': 'wind_speed_day_max',
        'mean_snow_surf_temp': 'snow_surf_temp_day_mean',
        'mean_snow_ground_temp': 'snow_ground_temp_day_mean'
    }

    # Create new DataFrame for weather data and stations:
    weather_data = pd.DataFrame(index=acc_df.index, columns=weather_columns.keys())
    used_station_codes = pd.Series(index=acc_df.index, dtype=object)

    # Iterate over all accident data:
    for i, accident in acc_df.iterrows():
        accident_date = accident['date']

        # Filter weather data by accident date:
        same_day_measurements = hist_measure_df[hist_measure_df['measure_date'] == accident_date]

        if same_day_measurements.empty:
            continue

        # Scaling of latitude, longitude, and elevation:
        scaled_lat_lon = same_day_measurements[['imis_longitude', 'imis_latitude']] * lat_lon_weight
        scaled_elevation = same_day_measurements['imis_elevation'] * elevation_weight
        scaled_coords = pd.concat([scaled_lat_lon, scaled_elevation], axis=1)

        # Train the K-NN model with the geographical data (Longitude, Latitude, Elevation):
        knn = NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
        knn.fit(scaled_coords)

        # Define the accident point (Longitude, Latitude, Elevation):
        accident_point = pd.DataFrame([[accident['start_zone_coordinates_longitude'] * lat_lon_weight,
                                        accident['start_zone_coordinates_latitude'] * lat_lon_weight,
                                        accident['start_zone_elevation'] * elevation_weight
                                        ]],
                                      columns=['imis_longitude', 'imis_latitude', 'imis_elevation'])

        # Find the nearest neighbors:
        distances, indices = knn.kneighbors(accident_point)

        # Filter valid neighbors:
        valid_indices = [idx for idx in indices[0] if idx < len(same_day_measurements)]
        num_neighbors = min(3, len(valid_indices))

        if num_neighbors == 0:
            continue

        neighbor_indices = valid_indices[:num_neighbors]

        # Calculate the mean values of the weather data:
        neighbor_data = same_day_measurements.iloc[neighbor_indices]
        mean_values = neighbor_data[list(weather_columns.values())].mean()

        # If NaN values are present, try with the next 6 neighbors:
        if np.isnan(mean_values.mean()) and len(valid_indices) >= 6:
            num_neighbors = min(6, len(valid_indices))
            neighbor_indices = valid_indices[:num_neighbors]
            neighbor_data = same_day_measurements.iloc[neighbor_indices]
            mean_values = neighbor_data[list(weather_columns.values())].mean()

        # Assign the mean values to the corresponding column:
        for col, source_col in weather_columns.items():
            weather_data.at[i, col] = mean_values.get(source_col, np.nan)

        # Save the used stations:
        used_station_codes.at[i] = ', '.join(neighbor_data['station_code'].astype(str))

    # Add the column for the used stations to the weather data DataFrame:
    weather_data['used_station_codes'] = used_station_codes

    # Combine the original accident data with the weather data:
    acc_mapped_weather_df = pd.concat([acc_df, weather_data], axis=1)
    return acc_mapped_weather_df

In [4]:
acc_mapped_weather_df = find_closest_weather_stations(acc_df, hist_measure_df, 1.0, 0.1)
acc_mapped_weather_df.to_csv('assets/01_3_SLF_hist_mapped_avalanche_accidents.csv', index=False)
print(acc_mapped_weather_df.head())
print(f"Runtime: {timer()-start:.2f} s")

         date canton municipality  start_zone_coordinates_latitude  \
0  1998-01-04     GR     Langwies                        46.825069   
1  1998-01-04     UR    Andermatt                        46.617104   
2  1998-01-08     GR        Davos                        46.733421   
3  1998-01-09     GR        Davos                        46.783051   
4  1998-01-10     GR      Fideris                        46.852684   

   start_zone_coordinates_longitude  start_zone_elevation  number_dead  \
0                          9.790353                2500.0            0   
1                          8.616889                2400.0            1   
2                          9.817899                2560.0            0   
3                          9.774500                2440.0            0   
4                          9.724397                2380.0            0   

   number_caught  number_fully_buried mean_air_temp mean_wind_speed  \
0              1                    0      -5.69375        5.12